### 4.4.1 导入模块并制定模型输入维度

In [31]:
# 导入声明
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam

In [32]:
# 指定模型输入维度：图像尺寸和噪声向量z的长度：
img_rows = 28
img_cols = 28
channels = 1

# 输入图像的维度
img_shape = (img_rows, img_cols, channels)

# 输入生成器的噪声向量的大小
z_dim = 100

### 4.4.2 构造生成器

In [33]:
def build_generator(z_dim):
    model = Sequential()

    # 通过全连接层将输入重新调整大小为7×7×256的张量
    model.add(Dense(256 * 7 * 7, input_dim = z_dim))
    model.add(Reshape((7, 7, 256)))

    # 转置卷积层从大小为7×7×256的张量到14×14×128的张量
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))

    # 批归一化
    model.add(BatchNormalization())

    # LeakyReLU激活函数
    model.add(LeakyReLU(alpha=0.01))

    # 转置卷积层从大小为14×14×128的张量到14×14×64的张量
    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))

    # 批归一化
    model.add(BatchNormalization())

    # LeakyReLU激活函数
    model.add(LeakyReLU(alpha=0.01))

    # 转置卷积层从大小为14×14×64的张量到28×28×1的张量
    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))

    # 带tanh激活函数的输出层
    model.add(Activation('tanh'))

    return model

### 4.4.3 构造鉴别器

In [34]:
def build_discriminator(img_shape):

    model = Sequential()

    # 卷积层，从大小为28×28×1的张量到14×14×32的张量
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))

    # LeakyReLU激活函数
    model.add(LeakyReLU(alpha=0.01))

    # 卷积层，从大小14×14×32的张量到7×7×64的张量
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))

    # 批归一化
    model.add(BatchNormalization())

    # LeakyReLU激活函数
    model.add(LeakyReLU(alpha=0.01))

    # 卷积层，从大小7×7×64的张量到3×3×128的张量
    model.add(Conv2D(128, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))

    # 批归一化
    model.add(BatchNormalization())

    # LeakyReLU激活函数
    model.add(LeakyReLU(alpha=0.01))

    # 带sigmoid激活函数的输出层
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

### 4.4.4　构建并运行DCGAN

In [35]:
def build_gan(generator, discriminator):
    model = Sequential()

    # 生成器和鉴别器结合成一个模型
    model.add(generator)
    model.add(discriminator)

    return model

In [36]:
# 构建并编译鉴别器
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
                                            optimizer=Adam(),
                                            metrics=['accuracy'])

# 构建生成器
generator = build_generator(z_dim)

# 生成器训练时鉴别器参数保持不变
discriminator.trainable = False

# 构建并编译鉴别器固定的GAN模型来训练生成器
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [37]:
# 训练DCGAN
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):

    # 加载MNIST数据
    (X_train, _), (_, _) = mnist.load_data()

    # 灰度像素值从[0，255]缩放到[1，1]
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3)

    # 真实图像标签为1
    real = np.ones((batch_size, 1))

    # 伪图像标签为0
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        # -------------------------
        #  Train the Discriminator
        # -------------------------

        # 获取一批真实图像
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # 生成一批伪图像
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        # 训练鉴别器
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train the Generator
        # ---------------------

        # 生成一批伪图像
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        # 训练生成器
        g_loss = gan.train_on_batch(z, real)

        if(iteration + 1) % sample_interval == 0:

            # Save losses and accuracies so they can be plotted after training
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # 输出训练过程
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % 
                        (iteration + 1, d_loss, 100.0 * accuracy, g_loss))
            
            # 输出生成图像的采样
            sample_images(generator)

In [38]:
def sample_images(generator, image_grid_rows = 4, image_grid_columns = 4):

    # 随机噪声采样
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # 从随机噪声生成图像
    gen_imgs = generator.predict(z)

    # 图像像素缩放到[0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # 设置图像网格
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns,
                                                    figsize=(4, 4),
                                                    sharey=True,
                                                    sharex=True)
    
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # 输出图像网格
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

In [ ]:
# 运行模型

# 设置超参数
iterations = 20000
batch_size = 128
sample_interval = 1000

# 训练DCGAN直到制定的迭代次数
train(iterations, batch_size, sample_interval)

In [ ]:
losses = np.array(losses)

# Plot training losses for Discriminator and Generator
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, losses.T[0], label="Discriminator loss")
plt.plot(iteration_checkpoints, losses.T[1], label="Generator loss")

plt.xticks(iteration_checkpoints, rotation=90)

plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracies = np.array(accuracies)

# Plot Discriminator accuracy
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, accuracies, label="Discriminator accuracy")

plt.xticks(iteration_checkpoints, rotation=90)
plt.yticks(range(0, 100, 5))

plt.title("Discriminator Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy (%)")
plt.legend()